# 实现 任务参数 记录

## 数据存储与写入

In [13]:
import yaml
# data_dict = [{'name': 'John Doe', 'occupation': 'gardener'}]
data_dict = []
# 清空与写入
def write_data(data_dict):
    with open(os.path.join('tmp_data','data.yaml'), 'w') as f:
        data = yaml.dump(data_dict, f)

# 数据读取与写入      
def get_data():
    with open(os.path.join('tmp_data','data.yaml')) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    return data 

## 定义接口获取数据

### 选择摄像头-ID号
- 可选（1，2，3，4），分别对应不同位置的四个摄像头

In [ ]:
@app.route('/get_camera', methods=["POST"])
def get_camera():
    camera_id = request.values.get("camera_id")
    return camera_id

### 确定检测任务
- 行人检测 -0
- 行人重识别 -1
- 行人属性检测 -2
- 口罩检测 -3
- 摔倒检测 -4
- 抽烟检测 -5


In [ ]:
@app.route('/get_opt', methods=["POST"])
def get_opt():
    opt_id = request.values.get("opt_id")
    return opt_id

### 选择追踪目标

#### 客户端上传视频
- 上传格式 .mp4

In [ ]:
@app.route('/get_tar_video', methods=["POST"])
def get_tar_video():
    # 获取上传的视频
    video = request.files.get('video')
    # 文件存储 - 此处注意传输过程中图像选用流式传输，基于二进制形式实现
    with open(os.path.join('tmp_data','tar_video.mp4'), 'wb') as f:
        f.write(data)
    return 0

#### 确定追踪目标
- 通过运行AI算法对上传的视频中的所有人物进行id分配和裁图存储处理
- 将所有人物的图像保存到了指定文件夹中
- 视野中每个人分配一个文件夹进行图片的保存
- 前后端部分展示各个文件夹中的裁图即可

In [ ]:
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/leon_FC')
from Leon_FC import *

def get_tar_crop():
    # 运行指定文件
    path_0 = leon_path_in(r'yolov5_track/track.py')
    path_1 = leon_path_in(r'yolov5_track/yolov5/weights/crowdhuman_yolov5m.pt')
    path_2 = leon_path_in(r'yolov5_track/strong_sort/configs/strong_sort.yaml')
    path_3 = os.path.join('tmp_data','tar_video.mp4')
    com = r'python {}  --yolo-weights {} --config-strongsort {}  --classes 0 --source {}'.format(path_0,path_1,path_2,path_3)
    # print(com)
    os.system(com)

# 相关功能函数初始化

## 指定图像的保存
- 对人物的类型分配为两种：['person','tar_person'] （是否为目标人物）
- 通过 数据库 为前后端返回指定数据

In [ ]:
import time
def crop_save(tar_cls,output):
    for key,unit_mes in output['crop'].items():
        cls = unit_mes['cls']
        if cls == tar_cls:
            crop_img = unit_mes['crop_img']
            dir_path = os.path.join('view_data',cls)
            os.makedirs(dir_path, exist_ok=True)
            cv2.imwrite(os.path.join(dir_path,str(time.time()+'.jpg')),crop_img)

## 数据库上传
- 将可视化数据上传至数据库供前后端进行获取与调用
- 此处 主要进行 数据库创建 | 相关表格创建 | 数据插入与更新等操作
- 具体存储使用上传 .zip 文件实现，前后端下载到指定位置后自行解压与处理

### 数据库创建

### 数据库 - 表创建

### 数据库 - 数据插入

## 图像标注

In [1]:
import cv2
# cv2-图像 [x1,y1,x2,y2]-坐标框 贴图文本
def draw_img(img,bbox,text):
    image = img.copy()
    # 左上角
    start_point = (int(bbox[1]),int(bbox[0]))
    # 右下角
    end_point = (int(bbox[3]),int(bbox[2]))
    # draw the rectangle
    cv2.rectangle(image, start_point, end_point, (0, 0, 255), thickness= 3, lineType=cv2.LINE_8) 
    #org: Where you want to put the text
    org = start_point
    # write the text on the input image
    cv2.putText(image, text, org, fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 1.8, color = (0,0,250))
    # display the output
    return image

## 行人检测

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Oct  1 13:22:46 2022

@author: leonzion
"""

from tools.yolo_leon_person import *
from PIL import Image
import datetime
from torchvision.ops import nms, roi_align, roi_pool
import torch.nn.functional as F
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
# 初始化检测器
yo = Yolo_infer_person()
img = Image.open(r'D:\Leon-Coding\Leon_TestData\mans_2.jpg')
yolo_output = yo.infer(img)
print(get_dict_dirs(yolo_output))


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tools/leon_yolo.pth model, and classes loaded.
Configurations:
----------------------------------------------------------------------
|                     keys |                                   values|
----------------------------------------------------------------------
|               model_path |                      tools/leon_yolo.pth|
|             classes_path |                            tools/cls.txt|
|              input_shape |                               [960, 960]|
|                      phi |                                Leon_nano|
|               confidence |                                      0.5|
|                  nms_iou |                                      0.3|
|          letterbox_image |                                     True|
|                     cuda |                                     True|
----------------------------------------------------------------------
	- img value_type:PIL.JpegImagePlugin.JpegImageFile
	+ crop
	|   + 0
	|   |   - crop_

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## 指定人物追踪

### 逐一获取行人数据

In [3]:
yolo_mes = yolo_output['crop']

### 重识别的函数设置

In [20]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Oct  2 12:58:46 2022

@author: leonzion
"""
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
from torchvision import datasets, models, transforms

import torch
from PIL import Image
import numpy as np
import cv2
import torch.nn.functional as F
import os
leon_path_in(os.path.join('reid','Person_reID_baseline_pytorch-master','model','ft_ResNet50'))
leon_path_in(os.path.join('reid','Person_reID_baseline_pytorch-master'))

from torchvision import datasets, models, transforms
data_transforms = transforms.Compose([
        transforms.Resize((256,128), interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data_transforms_infer = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256,128), interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 基于图片获取特征
def get_fea(path):
    img = data_transforms(Image.open(path))
#     img = torch.tensor(img.transpose(2, 0, 1))
    img = torch.unsqueeze(img, dim = 0)
    img = torch.tensor(img).to(torch.float32)
    net = torch.load('D:/Leon-Coding/Leon-PublicPlacesManage-System/models/reid-effi_b0.pth_net')
    outputs = net(img) 
    # ---- L2-norm Feature ------
    ff = outputs.data.cpu()
    fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
    ff = ff.div(fnorm.expand_as(ff))
    return ff

def get_reidnet(net_path):
    # 'D:/Leon-Coding/Leon-PublicPlacesManage-System/models/reid-effi_b0.pth_net'
    return torch.load(net_path)

def get_fig(net,t):
    # 图像数据转化
    t = torch.unsqueeze(torch.tensor(data_transforms_infer(t)), dim = 0)
    fig = net(t.to(torch.float32))
    # ---- L2-norm Feature ------
    ff = fig.data.cpu()
    fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
    ff = ff.div(fnorm.expand_as(ff))
    return ff

### 目标人物特征提取

In [8]:
net = get_reidnet('D:/Leon-Coding/Leon-PublicPlacesManage-System/models/reid-effi_b0.pth_net')
fig_tar = get_fea(r'D:\Leon-Coding\Leon_TestData\human-22.png')

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


### 群体特征提取与比较

In [9]:
for key,val in yolo_mes.items():
    crop_img = val['crop_img']
    fig = get_fig(net,crop_img)
    yolo_output['crop'][key]['fig'] = fig
    yolo_output['crop'][key]['score'] = F.cosine_similarity(fig,fig_tar,dim=1)

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


### 获取最高相似度

In [10]:
img_orig = np.array(yolo_output['img_orig'][0])
print(type(img_orig))
for key,val in yolo_mes.items():
    crop_img = val['crop_img']
    fig = get_fig(net,crop_img)
    yolo_output['crop'][key]['fig'] = fig
    yolo_output['crop'][key]['score'] = F.cosine_similarity(fig,fig_tar,dim=1)
# 标注最高分
score_list = []
for key,val in yolo_mes.items():
    score_list.append(yolo_output['crop'][key]['score'])

key = str(score_list.index(max(score_list)))
img_orig = draw_img(img_orig,yolo_output['crop'][key]['crop_bbox'],str(yolo_output['crop'][key]['score']))
cv2.imwrite('c.jpg',img_orig)

<class 'numpy.ndarray'>


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


True

## 行人属性识别

### 基本函数

In [29]:
# -*- coding: utf-8 -*-
"""
Created on Thu Oct  6 21:05:24 2022

@author: leonz
"""

import os
import json
import torch
from PIL import Image

import argparse
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
print(leon_path_in('passer-attribute_get'))
from net import get_model

from torchvision import datasets, models, transforms
######################################################################
# Argument
# ---------
parser = argparse.ArgumentParser()

parser.add_argument('--image_path', default=r'D:\Leon-Coding\Leon-PublicPlacesManage-System\passer-attribute_get\test_sample\test_7.jpg',help='Path to test image')
parser.add_argument('--dataset', default='market', type=str, help='dataset')
parser.add_argument('--backbone', default='resnet50', type=str, help='model')
parser.add_argument('--use-id', action='store_true', help='use identity loss')
args = parser.parse_known_args()[0]
assert args.dataset in ['market', 'duke']
assert args.backbone in ['resnet50', 'resnet34', 'resnet18', 'densenet121']

######################################################################
# Settings
# ---------
dataset_dict = {
    'market'  :  'Market-1501',
    'duke'  :  'DukeMTMC-reID',
}
num_cls_dict = { 'market':30, 'duke':23 }
num_ids_dict = { 'market':751, 'duke':702 }


######################################################################
# Model and Data
# ---------
def load_network(network):
    save_path = os.path.join('models','passer_attribute.pth')
    network.load_state_dict(torch.load(save_path))
    print('Resume model from {}'.format(save_path))
    return network

def load_image(path):
    src = Image.open(path)
    src = transforms(src)
    src = src.unsqueeze(dim=0)
    return src


def leon_get_attrmodel():
######################################################################
    model_name = '{}_nfc_id'.format(args.backbone) if args.use_id else '{}_nfc'.format(args.backbone)
    num_label, num_id = num_cls_dict[args.dataset], num_ids_dict[args.dataset]
    model = get_model(model_name, num_label, use_id=args.use_id, num_id=num_id)
    model = load_network(model)
    model.eval()
    return model


######################################################################
# Inference
# ---------
class predict_decoder(object):

    def __init__(self, dataset):
        with open(leon_path_in(os.path.join(r'passer-attribute_get','doc','label.json')), 'r') as f:
            self.label_list = json.load(f)[dataset]
        with open(leon_path_in(os.path.join(r'passer-attribute_get','doc','attribute.json')), 'r') as f:
            self.attribute_dict = json.load(f)[dataset]
        self.dataset = dataset
        self.num_label = len(self.label_list)

    def decode(self, pred):
        pred = pred.squeeze(dim=0)
        for idx in range(self.num_label):
            name, chooce = self.attribute_dict[self.label_list[idx]]
            if chooce[pred[idx]]:
                print('{}: {}'.format(name, chooce[pred[idx]]))

D:\Leon-Coding\Leon-PublicPlacesManage-System\passer-attribute_get


In [35]:

attri_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size=(288, 144)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
def get_attribute(model,img):
    src = attri_transforms(img).unsqueeze(dim=0)
    src = model.forward(src)
    pred = torch.gt(src, torch.ones_like(src)/2 )  # threshold=0.5
    Dec = predict_decoder(args.dataset)
    Dec.decode(pred)
    return pred

### 实际推理

In [36]:
model = leon_get_model()
for key,val in yolo_mes.items():
    crop_img = val['crop_img']
    fig = get_fig(net,crop_img)
    yolo_output['crop'][key]['attri'] = get_attribute(model,yolo_output['crop'][key]['crop_img'])
    print(yolo_output['crop'][key]['attri'])
    print('----')

Resume model from models\passer_attribute.pth


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


age: teenager
carrying backpack: no
carrying bag: no
carrying handbag: no
type of lower-body clothing: pants
length of lower-body clothing: long lower body clothing
sleeve length: short sleeve
hair length: long hair
wearing hat: no
gender: female
tensor([[False,  True, False, False, False, False, False,  True, False,  True,
          True, False,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])
----
age: teenager
carrying backpack: no
carrying bag: no
carrying handbag: no
type of lower-body clothing: pants
length of lower-body clothing: short
sleeve length: short sleeve
hair length: short hair
wearing hat: no
gender: male
tensor([[False,  True, False, False, False, False, False,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])
----
age: teenager
carrying backpack: no
ca

age: teenager
carrying backpack: no
carrying bag: no
carrying handbag: no
type of lower-body clothing: pants
length of lower-body clothing: long lower body clothing
sleeve length: short sleeve
hair length: long hair
wearing hat: no
gender: male
color of upper-body clothing: green
tensor([[False,  True, False, False, False, False, False,  True, False,  True,
          True, False, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False, False]])
----
age: teenager
carrying backpack: no
carrying bag: no
carrying handbag: no
type of lower-body clothing: pants
length of lower-body clothing: long lower body clothing
sleeve length: short sleeve
hair length: short hair
wearing hat: no
gender: female
tensor([[False,  True, False, False, False, False, False,  True, False,  True,
         False, False,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, Fals

## 其他 - 基础检测功能
- 分为三个功能
    - 吸烟检测
    - 摔倒检测
        - 定义摔倒不起检测功能
    - 口罩检测
- 三个功能来自同一个检测模型
    - 需要做一个任务分流函数（基于模型输出进行筛选性操作）

### 模型初始化

In [ ]:
def get_yolo_relation():
    pass

### 定义类别映射字典

In [44]:
class_dict = {'smoke':['smoke'],'fall':['fall'],'mask':['face_mask','mask']}
# for key,value in class_dict.items():
#     print(key)
# output: smoke <br> fall <br> mask

### 定义模型检测-输出筛选函数
- 只输出当前任务的检测结果

In [57]:
def choose_opt(mode,output,class_dict):
    # 待删除数据的 键 存储器 - 限于Python字典的性质限制
    mid_list = []
    # 基于类别进行筛选
    for key,unit_mes in output['crop'].items():
        cls = unit_mes['cls']
        # 定义目标类别的变量域
        tar_list = class_dict[mode]
        # 筛选操作
        if cls not in tar_list:
            mid_list.append(key)
    for key in mid_list:
        output['crop'].pop(key, None)
    return output

# test_dict = {'crop':{'smoke':{'cls':'smoke'},'fall':{'cls':'fall'},'mask':{'cls':'mask'}}}
# choose_opt('smoke',test_dict,class_dict)
# output: {'crop': {'smoke': {'cls': 'smoke'}}}

# 图像源 初始化

In [1]:
import cv2
 
class VideoCamera(object):
    def __init__(self):
        # 定义视频数据的图像来源
        self.video = cv2.VideoCapture(0)
    
    def __del__(self):
        self.video.release()
    
    # 设置获取视频的图像数据
    def get_frame(self):
        # 每次运行都通过来源进行数据获取
        success, image = self.video.read()
        # 将图片进行编码，转化为指定的图片格式
        ret, jpeg = cv2.imencode('.jpg', image)
        # 通过字节形式进行流传输
        return jpeg.tobytes()

# 函数功能汇总
- 用于确定选择该项目所选择的AI处理算法

In [5]:
def get_opt():
    pass

# 图像传输

In [2]:
import os
from flask import Flask, render_template, Response, make_response
app = Flask(__name__)
 
# 相机推流 - 创建对象 - 根据图生成二进制传输流
def gen(camera):
    while True:
        frame = camera.get_frame()
        # 生成二进制传输流 同时生成包含头部数据和图片数据的传输流
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

# 每次访问，即返回一张处理好的照片 - 注意此处的response是用来进行数据的 反馈操作
@app.route('/video_feed')
def video_feed():
    return Response(gen(VideoCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
 
#当前实时相机画面
@app.route('/cur_camera')
def cur_camera():
    return render_template(r'Flask-StreamData-Trans.html')

In [3]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.7.8:5000
Press CTRL+C to quit
192.168.7.8 - - [08/Oct/2022 22:41:19] "GET / HTTP/1.1" 404 -
192.168.7.8 - - [08/Oct/2022 22:41:19] "GET /favicon.ico HTTP/1.1" 404 -
192.168.7.8 - - [08/Oct/2022 22:41:38] "GET //video_feed HTTP/1.1" 200 -
192.168.7.8 - - [08/Oct/2022 22:41:41] "GET /video_feed HTTP/1.1" 200 -
